# 楽曲のコード進行を取得したい。

## Songleを用いたやり方  

https://songle.jp/ にはAPIが公開されている  
http://widget.songle.jp/docs/v1 を読むと、  
http://widget.songle.jp/api/v1/song/chord.json?url=www.youtube.com/watch?v=PqJNc9KVIZE  
のようにすればコード進行のjsonを取得できるようだ。

REST API なのでHTTPメソッドでアクセスすればよく、pythonからはrequestsでデータを取得可能。  
https://note.nkmk.me/python-requests-web-api/

In [4]:
import requests

In [7]:
song_url = "www.youtube.com/watch?v=LtSNzPyo0lA"
# ぼくらはみんな意味不明 ピノキオピー

r_get = requests.get("http://widget.songle.jp/api/v1/song/chord.json?url=" + song_url)

print(r_get.status_code)

200


In [8]:
r_get.headers

{'Server': 'nginx', 'Date': 'Wed, 19 Dec 2018 15:16:05 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Status': '200 OK', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Access-Control-Allow-Origin': '*', 'ETag': 'W/"d78bac811763b859e90b56a77a7894c4"', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Runtime': '0.380415', 'X-Request-Id': '8e269ab0-b21a-461b-aad1-4c155acab185', 'Content-Encoding': 'gzip'}